In [0]:
!unzip trainrotfaces.zip

Streaming output truncated to the last 5000 lines.
  inflating: train.rotfaces/train/81-860581_1921-09-30_1992.jpg  
  inflating: train.rotfaces/train/81-8614381_1931-07-29_2012.jpg  
  inflating: train.rotfaces/train/81-8649881_1982-09-08_2009.jpg  
  inflating: train.rotfaces/train/81-871281_1981-03-02_2010.jpg  
  inflating: train.rotfaces/train/81-8854681_1958-09-05_2014.jpg  
  inflating: train.rotfaces/train/81-8875281_1988-06-06_2013.jpg  
  inflating: train.rotfaces/train/81-8920481_1927-04-03_1956.jpg  
  inflating: train.rotfaces/train/81-8986681_1967-09-25_1960.jpg  
  inflating: train.rotfaces/train/81-9078381_1987-06-12_2010.jpg  
  inflating: train.rotfaces/train/81-9154081_1902-03-23_1950.jpg  
  inflating: train.rotfaces/train/81-9233381_1974-12-24_2013.jpg  
  inflating: train.rotfaces/train/81-928681_1981-09-07_2012.jpg  
  inflating: train.rotfaces/train/81-9290681_1958-04-18_1987.jpg  
  inflating: train.rotfaces/train/81-93781_1949-03-02_2004.jpg  
  inflating: tra

In [0]:
!unzip testrotfaces.zip

Archive:  testrotfaces.zip
   creating: test.rotfaces/
   creating: test.rotfaces/test/
  inflating: test.rotfaces/test.truth.csv  
  inflating: test.rotfaces/test/90-1019890_1931-08-10_1978.jpg  
  inflating: test.rotfaces/test/90-10241990_1984-11-28_2007.jpg  
  inflating: test.rotfaces/test/90-102690_1966-09-09_2011.jpg  
  inflating: test.rotfaces/test/90-10303590_1983-01-26_2010.jpg  
  inflating: test.rotfaces/test/90-10311690_1947-04-14_1940.jpg  
  inflating: test.rotfaces/test/90-10504790_1986-07-14_2014.jpg  
  inflating: test.rotfaces/test/90-10506490_1966-03-23_2012.jpg  
  inflating: test.rotfaces/test/90-10520890_1968-11-15_2011.jpg  
  inflating: test.rotfaces/test/90-1060790_1925-06-20_1953.jpg  
  inflating: test.rotfaces/test/90-10704590_1977-06-16_2007.jpg  
  inflating: test.rotfaces/test/90-10840090_1903-06-26_1944.jpg  
  inflating: test.rotfaces/test/90-1084490_1987-07-31_2013.jpg  
  inflating: test.rotfaces/test/90-1095190_1957-09-04_2012.jpg  
  inflating: tes

In [0]:
import numpy as np
import pandas as pd
from PIL import Image
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

In [0]:
traindf = pd.read_csv("/content/train.rotfaces/train.truth.csv", dtype = str)
testdf = pd.read_csv("/content/test.rotfaces/test.truth.csv", dtype = str)

In [0]:
datagen = ImageDataGenerator(rescale = 1./255., validation_split = 0.20)

In [0]:
train_generator = datagen.flow_from_dataframe(
dataframe = traindf,
directory = "/content/train.rotfaces/train",
x_col = "fn",
y_col = "label",
subset = "training",
batch_size = 128,
seed = 42,
shuffle = True,
class_mode = "categorical",
target_size = (64,64))

Found 39117 validated image filenames belonging to 4 classes.


In [0]:
valid_generator = datagen.flow_from_dataframe(
dataframe = traindf,
directory = "/content/train.rotfaces/train",
x_col = "fn",
y_col = "label",
subset = "validation",
batch_size = 128,
seed = 42,
shuffle = True,
class_mode="categorical",
target_size = (64,64))

Found 9779 validated image filenames belonging to 4 classes.


In [0]:
test_datagen = ImageDataGenerator(rescale = 1./255.)

In [0]:
test_generator = test_datagen.flow_from_dataframe(
dataframe = testdf,
directory = "/content/test.rotfaces/test",
x_col = "fn",
y_col = None,
batch_size = 70,
seed = 42,
shuffle = False,
class_mode = None,
target_size = (64,64))

Found 700 validated image filenames.


In [0]:
num_classes = 4

model = tf.keras.models.Sequential() 

model.add(tf.keras.layers.Conv2D(32, (3, 3), padding='same',input_shape=(64,64,3)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Conv2D(32, (3, 3)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Conv2D(64, (3, 3), padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Conv2D(64, (3, 3)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
model.compile(tf.keras.optimizers.RMSprop(lr=0.001), loss="categorical_crossentropy",metrics=["accuracy"])

In [0]:
STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n // valid_generator.batch_size
STEP_SIZE_TEST = test_generator.n // test_generator.batch_size 

In [0]:
model.fit_generator(generator = train_generator,
                    steps_per_epoch = STEP_SIZE_TRAIN,
                    validation_data = valid_generator,
                    validation_steps = STEP_SIZE_VALID,
                    epochs = 10
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
305/305 [==============================] - 23s 75ms/step - loss: 0.4446 - accuracy: 0.8258 - val_loss: 0.1214 - val_accuracy: 0.9554
Epoch 2/10
305/305 [==============================] - 22s 72ms/step - loss: 0.1105 - accuracy: 0.9623 - val_loss: 0.0895 - val_accuracy: 0.9689
Epoch 3/10
305/305 [==============================] - 22s 72ms/step - loss: 0.0690 - accuracy: 0.9755 - val_loss: 0.0959 - val_accuracy: 0.9669
Epoch 4/10
305/305 [==============================] - 22s 71ms/step - loss: 0.0425 - accuracy: 0.9849 - val_loss: 0.0541 - val_accuracy: 0.9812
Epoch 6/10
305/305 [==============================] - 22s 71ms/step - loss: 0.0367 - accuracy: 0.9875 - val_loss: 0.0602 - val_accuracy: 0.9812
Epoch 7/10
305/305 [==============================] - 22s 71ms/step - loss: 0.0311 - accuracy: 0.9898 - val_loss: 0.0681 - val_accuracy: 0.9801
Epoch 8/10
305/305 [==============================] - 22s 71

In [0]:
model.evaluate_generator(generator=valid_generator,
steps=STEP_SIZE_TEST, verbose=1)

Instructions for updating:
Please use Model.evaluate, which supports generators.
10/10 [==============================] - 0s 43ms/step - loss: 0.0796 - accuracy: 0.9812


[0.07956865429878235, 0.981249988079071]

In [0]:
test_generator.reset()
pred = model.predict_generator(test_generator,
steps = STEP_SIZE_TEST,
verbose=1)

Instructions for updating:
Please use Model.predict, which supports generators.
10/10 [==============================] - 0s 21ms/step


In [0]:
predicted_class_indices=np.argmax(pred,axis=1)

In [0]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [0]:
filenames=test_generator.filenames
results=pd.DataFrame({"fn":filenames,
                      "label":predictions})
results.to_csv("/content/test.rotfaces/test.preds.csv",index=False)

In [0]:
''' As a test set I used 700 images from the original test set. 
I used a small portion because I did not find it feasible to manually label all test images
for later comparison with the outputs predicted by the network. '''

ROTATE_LEFT = -90
ROTATE_RIGHT = -270
UPSIDE_DOWN = -180
UPRIGHT = -360

path_save = "/content/corrected.images"
path_open = "/content/test.rotfaces/test"

files_pred = pd.read_csv("/content/test.rotfaces/test.preds.csv")

labels = files_pred['label']
fns = files_pred['fn']

for i in range(0,len(labels)):
    
    im = Image.open(os.path.join(path_open,str(fns[i])))
    if(labels[i] == "rotated_left"):
        rotated = im.rotate(ROTATE_LEFT)
        rotated.save(os.path.join(path_save,str(fns[i])))
    elif(labels[i] == "rotated_right"):
        rotated = im.rotate(ROTATE_RIGHT)
        rotated.save(os.path.join(path_save,str(fns[i])))
    elif(labels[i] == "UPSIDE_DOWN"):
        rotated = im.rotate(-180)
        rotated.save(os.path.join(path_save,str(fns[i])))
    elif(labels[i] == "upright"):
        rotated = im.rotate(UPRIGHT)
        rotated.save(os.path.join(path_save,str(fns[i])))
        


In [0]:
''' The 700 images of the test set had their orientations 
corrected according to the prediction of the network and were saved in an array numpy. '''

path_imgs_corrected = "/content/corrected.images" 

list_corrected_images = os.listdir(path_imgs_corrected)

imgs = []

for fname in list_corrected_images:
    img = Image.open(os.path.join(path_imgs_corrected,str(fname)))
    img = np.asarray(img)
    imgs.append(img)

my_array_images = np.array(imgs)


In [25]:
!zip -r images corrected.images

  adding: corrected.images/ (stored 0%)
  adding: corrected.images/90-33088990_1984-10-29_2010.jpg (deflated 8%)
  adding: corrected.images/90-13969890_1985-02-18_2012.jpg (deflated 7%)
  adding: corrected.images/90-26609690_1984-01-16_2013.jpg (deflated 8%)
  adding: corrected.images/90-35591890_1983-04-28_2014.jpg (deflated 7%)
  adding: corrected.images/90-40780290_1932-09-12_1960.jpg (deflated 9%)
  adding: corrected.images/91-1424791_1947-05-14_1969.jpg (deflated 9%)
  adding: corrected.images/90-36910590_1993-12-22_2013.jpg (deflated 8%)
  adding: corrected.images/90-22737890_1985-05-16_2010.jpg (deflated 9%)
  adding: corrected.images/90-41046290_1991-09-20_2014.jpg (deflated 8%)
  adding: corrected.images/91-10093491_1982-04-15_2010.jpg (deflated 10%)
  adding: corrected.images/90-17190_1940-01-17_2014.jpg (deflated 7%)
  adding: corrected.images/90-164390_1940-01-22_2015.jpg (deflated 8%)
  adding: corrected.images/90-325590_1933-04-01_1996.jpg (deflated 13%)
  adding: correct